In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

/home/djjin/anaconda3/envs/conda_visual_HPE/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0
torch:  1.13 ; cuda:  cu117
detectron2: 0.6


In [2]:
import os
import sys
import logging
import argparse

pth = '/'.join(sys.path[0].split('/')[:-1])
sys.path.insert(0, pth)

from pprint import pprint
from PIL import Image
from copy import deepcopy
import numpy as np
np.random.seed(1)

home_dir = os.path.abspath(os.getcwd()+"/../")
sys.path.append(home_dir)
print(home_dir)

import warnings
warnings.filterwarnings(action='ignore')
logger = logging.getLogger(__name__)

import torch
from torchvision import transforms
from torch.nn import functional as F

from detectron2.data import MetadataCatalog
from detectron2.utils.colormap import random_color
from detectron2.structures import Boxes, ImageList, Instances, BitMasks, BoxMode

from hdecoder.BaseModel import BaseModel
from hdecoder import build_model

from utils.arguments import load_opt_command
from utils.misc import hook_metadata, hook_switcher, hook_opt
from utils.distributed import init_distributed
from utils.arguments import load_opt_from_config_files, load_config_dict_to_opt

/home/djjin/Mygit/X-Decoder


Invalid MIT-MAGIC-COOKIE-1 key

In [3]:
parser = argparse.ArgumentParser(description='Pretrain or fine-tune models for NLP tasks.')
parser.add_argument('--command', default="evaluate", help='Command: train/evaluate/train-and-evaluate')
parser.add_argument('--conf_files', nargs='+', help='Path(s) to the config file(s).')
parser.add_argument('--user_dir', help='Path to the user defined module for tasks (models, criteria), optimizers, and lr schedulers.')
parser.add_argument('--config_overrides', nargs='*', help='Override parameters on config with a json style string, e.g. {"<PARAM_NAME_1>": <PARAM_VALUE_1>, "<PARAM_GROUP_2>.<PARAM_SUBGROUP_2>.<PARAM_2>": <PARAM_VALUE_2>}. A key with "." updates the object in the corresponding nested dict. Remember to escape " in command line.')
parser.add_argument('--overrides', help='arguments that used to override the config file in cmdline', nargs=argparse.REMAINDER)

cmdline_args = parser.parse_args('')
# cmdline_args.conf_files = [os.path.join(home_dir, "configs/xdecoder/svlp_focalt_lang.yaml")]
cmdline_args.conf_files = [os.path.join(home_dir, "configs/xdecoder/vcoco.yaml")]
cmdline_args.overrides = ['WEIGHT', '../checkpoints/xdecoder_focalt_best_openseg.pt'] 
cmdline_args.overrides

opt = load_opt_from_config_files(cmdline_args.conf_files)

keys = [cmdline_args.overrides[idx*2] for idx in range(len(cmdline_args.overrides)//2)]
vals = [cmdline_args.overrides[idx*2+1] for idx in range(len(cmdline_args.overrides)//2)]
vals = [val.replace('false', '').replace('False','') if len(val.replace(' ', '')) == 5 else val for val in vals]
types = []
for key in keys:
    key = key.split('.')
    ele = opt.copy()
    while len(key) > 0:
        ele = ele[key.pop(0)]
    types.append(type(ele))

config_dict = {x:z(y) for x,y,z in zip(keys, vals, types)}
config_dict

load_config_dict_to_opt(opt, config_dict)
for key, val in cmdline_args.__dict__.items():
    if val is not None:
        opt[key] = val
opt = init_distributed(opt)

In [4]:
model = BaseModel(opt, build_model(opt)).train().cuda()

## Piplene

In [5]:
# import importlib

# base_name = 'base_dir'
# base_path =  os.path.join(opt['base_path'], '__init__.py')
# spec = importlib.util.spec_from_file_location(base_name, base_path)
# module = importlib.util.module_from_spec(spec)
# sys.modules[base_name] = module
# spec.loader.exec_module(module)
# logger.info(f"Imported {base_name} at base_path {opt['base_path']}")

# pipeline_module = importlib.import_module(f"base_dir.pipeline.{opt['PIPELINE']}")
# pipeline_class = getattr(pipeline_module, opt['PIPELINE'])
# pipeline = pipeline_class(opt)

In [6]:
opt

{'PIPELINE': 'HDecoderPipeline',
 'TRAINER': 'hdecoder',
 'SAVE_DIR': '../../data/output/test',
 'base_path': '../',
 'RESUME': False,
 'WEIGHT': True,
 'RESET_DATA_LOADER': False,
 'RESUME_FROM': '',
 'PYLEARN_MODEL': '',
 'DONT_LOAD_MODEL': True,
 'LOG_EVERY': 500,
 'FIND_UNUSED_PARAMETERS': False,
 'FP16': False,
 'PORT': '36873',
 'LOADER': {'JOINT': True, 'KEY_DATASET': 'coco'},
 'VERBOSE': True,
 'MODEL': {'NAME': 'hoi_model',
  'HEAD': 'hoi_head',
  'MASK_ON': False,
  'KEYPOINT_ON': False,
  'LOAD_PROPOSALS': False,
  'DIM_PROJ': 512,
  'BACKBONE_DIM': 768,
  'BACKBONE': {'NAME': 'focal_dw',
   'PRETRAINED': '',
   'LOAD_PRETRAINED': False,
   'FOCAL': {'PRETRAIN_IMG_SIZE': 224,
    'PATCH_SIZE': 4,
    'EMBED_DIM': 96,
    'DEPTHS': [2, 2, 6, 2],
    'FOCAL_LEVELS': [3, 3, 3, 3],
    'FOCAL_WINDOWS': [3, 3, 3, 3],
    'DROP_PATH_RATE': 0.3,
    'MLP_RATIO': 4.0,
    'DROP_RATE': 0.0,
    'PATCH_NORM': True,
    'USE_CONV_EMBED': True,
    'SCALING_MODULATOR': True,
    'USE_CH

```sh
CUDA_VISIBLE_DEVICES=0 python entry.py train \
            --conf_files configs/xdecoder/segvlp_focalt_lang.yaml \
            --overrides \
            COCO.INPUT.IMAGE_SIZE 1024 \
            MODEL.DECODER.CAPTIONING.ENABLED True \
            MODEL.DECODER.RETRIEVAL.ENABLED True \
            MODEL.DECODER.GROUNDING.ENABLED True \
            MODEL.DECODER.CAPTIONING_WEIGHT 8 \
            MODEL.DECODER.RETRIEVAL_WEIGHT 8 \
            MODEL.DECODER.TOP_CAPTIONING_LAYERS 3 \
            MODEL.DECODER.TOP_RETRIEVAL_LAYERS 3 \
            MODEL.DECODER.TOP_GROUNDING_LAYERS 6 \
            COCO.TEST.BATCH_SIZE_TOTAL 1 \
            COCO.TRAIN.BATCH_SIZE_TOTAL 1 \
            COCO.TRAIN.BATCH_SIZE_PER_GPU 1 \
            VLP.TEST.BATCH_SIZE_TOTAL 32 \
            VLP.TRAIN.BATCH_SIZE_TOTAL 32 \
            VLP.TRAIN.BATCH_SIZE_PER_GPU 32 \
            MODEL.DECODER.HIDDEN_DIM 512 \
            MODEL.ENCODER.CONVS_DIM 512 \
            MODEL.ENCODER.MASK_DIM 512 \
            FP16 True \
            WEIGHT True \
            RESUME_FROM \pth\to\focalt_unicl_pretrain.pt
```

In [7]:
from trainer import HDecoder_Trainer as Trainer
trainer = Trainer(opt)
trainer.train()

----------------
MPI Adapter data
----------------
environment info: no MPI
init method url: tcp://127.0.0.1:36873
world size: 1
local size: 1
rank: 0
local rank: 0
master address: 127.0.0.1
master port: 36873
----------------


FileNotFoundError: [Errno 2] No such file or directory: ''